In [ ]:
import pandas as pd
import nltk

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/project/Sentence_Score.csv')
dataset = dataset.iloc[:, 1:]
dataset.head()

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   
2  FBIS4-56863  d53i  During the meeting, the Shining Path Central C...   
3  FBIS4-56863  d53i  The document entitled "To Grow Strong on the G...   
4  FBIS4-56863  d53i  A Shining Path pamphlet dated February, which ...   

                                      Clean_sentence  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   
2  During meet shin path cent commit decided cont...     19    123   
3  the document entitled to grow strong ground pa...     49    327   
4  a shin path pamphlet dated February sign lead ...     30    207   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   
1      0.004720       0.00000    0.000000            0.000         0.008805   
2      0.014380       0.00000    0.008929            0.016         0.014032   
3      0.022180       0.02439    0.026786            0.036         0.011038   
4      0.017933       0.00000    0.035714            0.020         0.012965   

   font_style_score  sentence_position  thematic_score  sentence_similarity  
0          0.032258                1.0        0.121429             0.351307  
1          0.000000                0.8        0.007143             0.601866  
2          0.000000                0.6        0.121429             0.647553  
3          0.000000                0.4        0.221429             0.688493  
4          0.000000                0.2        0.157143             0.812094

In [ ]:
st = dataset.iloc[:,6:].columns
def generate_summary():
  df = pd.DataFrame()
  for dir in set(dataset['Dir']):
    for doc in set(dataset[dataset['Dir'] == dir]['Docno']):
      temp_df = dataset[(dataset['Dir'] == dir) & (dataset['Docno'] == doc)]
      dic = {'Docno':doc, 'Dir': dir}
      for s in st:
        demo_df = temp_df.sort_values(by = s, ascending = False)
        # dic[s] = ". ".join(list(temp_df.nlargest(5, [s])['clean_sentence']))
        rs = ''
        cnt = 0
        for i in demo_df['sentence']:
          if cnt + len(i.split()) <= 100:
            cnt += len(i.split())
            rs += (i+' ')
        dic[s] = rs
      df = df.append(dic, ignore_index = True)
  return df
final_dataset = generate_summary()

In [ ]:
final_dataset.to_csv("/content/drive/MyDrive/project/Summary.csv")

In [ ]:
reference_dataset=pd.read_csv("/content/drive/MyDrive/project/reference_summary_2001.csv")
generate_dataset=pd.read_csv("/content/drive/MyDrive/project/Summary.csv")

In [ ]:
reference_dataset=reference_dataset.iloc[:,1:]
generate_dataset=generate_dataset.iloc[:,1:]

In [ ]:
generate_dataset.head(1)

Docno   Dir                                       tf-idf_score  \
0  AP880926-0203  d30e  Officials of the seven key industrialized nati...   

                                        adverb_score  \
0  ``It is ... sad to note that we, American, Asi...   

                                          verb_score  \
0  ``In a market place where one group of operato...   

                                     adjective_score  \
0  Officials of the seven key industrialized nati...   

                                     text_rank_score  \
0  Brazil, with $121 billion in foreign loans, ha...   

                                    font_style_score  \
0     Third World countries led by Brazil, thewor...   

                                   sentence_position  \
0     Third World countries led by Brazil, thewor...   

                                      thematic_score  \
0     Third World countries led by Brazil, thewor...   

                                 sentence_similarity  
0     Third World countries led by Brazil, thewor...

In [ ]:
reference_dataset.head(1)

dir dir_original           Docno  \
0  d12b        d12bb  WSJ870123-0101   

                                         ref_summary  Chars  Words  
0  President Reagan has made welfare reform a pri...    657     96

In [ ]:
dataset1 = pd.merge(generate_dataset, reference_dataset, on='Docno')

In [ ]:
dataset1 = dataset1.drop(['dir', 'dir_original'], axis = 1)

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
st = dataset.iloc[:,6:].columns

In [ ]:
def get_rouge_score(lst):
  dic = {'Docno':lst[0], 'Dir':lst[1]}
  for i in range(2, len(lst)-3):
    dic[st[i-2]] = rouge.get_scores(lst[i], lst[-3])
  return dic

In [ ]:
dataset1 = dataset1.apply(get_rouge_score, axis = 1)

In [ ]:
# df1 = pd.DataFrame()
# for i in dataset1.apply(get_rouge_score, axis = 1):
#   df1 = df1.append(i, ignore_index = True)

In [ ]:
df1 = pd.DataFrame()
dfr1 = pd.DataFrame()
dfr2 = pd.DataFrame()
dfrl = pd.DataFrame()

for i in dataset1:
  dic_r1 = {'Docno': i['Docno'], 'Dir' : i['Dir']}
  dic_r2 = {'Docno': i['Docno'], 'Dir' : i['Dir']}
  dic_rl = {'Docno': i['Docno'], 'Dir' : i['Dir']}
  for s in st:
    dic_r1[s+'_r'] = i[s][0]['rouge-1']['r']
    dic_r1[s+'_p'] = i[s][0]['rouge-1']['p']
    dic_r1[s+'_f'] = i[s][0]['rouge-1']['f']

    dic_r2[s+'_r'] = i[s][0]['rouge-2']['r']
    dic_r2[s+'_p'] = i[s][0]['rouge-2']['p']
    dic_r2[s+'_f'] = i[s][0]['rouge-2']['f']

    dic_rl[s+'_r'] = i[s][0]['rouge-l']['r']
    dic_rl[s+'_p'] = i[s][0]['rouge-l']['p']
    dic_rl[s+'_f'] = i[s][0]['rouge-l']['f']

  dfr1 = dfr1.append(dic_r1, ignore_index = True)
  dfr2 = dfr2.append(dic_r2, ignore_index = True)
  dfrl = dfrl.append(dic_rl, ignore_index = True)

In [ ]:
dfr1.to_csv('/content/drive/MyDrive/project/rouge-1_score.csv')
dfr2.to_csv('/content/drive/MyDrive/project/rouge-2_score.csv')
dfrl.to_csv('/content/drive/MyDrive/project/rouge-l_score.csv')

In [ ]:
final_score = pd.DataFrame()
dic_1 = {'method':'rouge-1'}
dic_2 = {'method':'rouge-2'}
dic_l = {'method':'rouge-l'}
for s in dfr1.iloc[:, 2:].columns:
  dic_1[s] = dfr1[s].mean()
  dic_2[s] = dfr2[s].mean()
  dic_l[s] = dfrl[s].mean()
final_score = final_score.append(dic_1, ignore_index = True)
final_score = final_score.append(dic_2, ignore_index = True)
final_score = final_score.append(dic_l, ignore_index = True)
final_score.to_csv('/content/drive/MyDrive/project/final_score.csv')

In [ ]:
# df1.head(1)

Docno   Dir                                       tf-idf_score  \
0  AP880926-0203  d30e  [{'rouge-1': {'r': 0.3157894736842105, 'p': 0....   

                                        adverb_score  \
0  [{'rouge-1': {'r': 0.3157894736842105, 'p': 0....   

                                          verb_score  \
0  [{'rouge-1': {'r': 0.25, 'p': 0.23170731707317...   

                                     adjective_score  \
0  [{'rouge-1': {'r': 0.3815789473684211, 'p': 0....   

                                     text_rank_score  \
0  [{'rouge-1': {'r': 0.32894736842105265, 'p': 0...   

                                    font_style_score  \
0  [{'rouge-1': {'r': 0.3026315789473684, 'p': 0....   

                                   sentence_position  \
0  [{'rouge-1': {'r': 0.3026315789473684, 'p': 0....   

                                      thematic_score  \
0  [{'rouge-1': {'r': 0.3684210526315789, 'p': 0....   

                                 sentence_similarity  
0  [{'rouge-1': {'r': 0.32894736842105265, 'p': 0...

In [ ]:
# df1.to_csv("/content/drive/MyDrive/project/rouge_score.csv")

##bleu score

In [ ]:
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def get_bleu_score(lst):
  dic = {'Docno':lst[0], 'Dir':lst[1]}
  for i in range(2, len(lst)-3):
    can_sent = word_tokenize(lst[i])
    ref_sent = [word_tokenize(lst[-3])]
    dic[st[i-2]] = sentence_bleu(ref_sent, can_sent, weights=(1, 0, 0, 0)) 
  return dic

In [ ]:
df1 = pd.DataFrame()
for i in dataset1.apply(get_bleu_score, axis = 1):
  df1 = df1.append(i, ignore_index = True)

In [ ]:
df1.to_csv("/content/drive/MyDrive/project/bleu_score.csv")

In [ ]:
w1 = 0.33
w2 = 0.33
w3 = 0.33
def make_combination(lst):
  m1 = lst[0] 
  m2 = lst[1] 
  m3 = lst[2] 
  final_score = m1*w1 + m2*w2 + m3*w3
  return final_score
dataset['combined_score'] = dataset[['adjective_score', 'sentence_position', 'thematic_score']].apply(make_combination, axis = 1)

In [ ]:
dataset.head(2)

Docno   Dir                                           sentence  \
0  FBIS4-56863  d53i  BFN 106> ["Exclusive" report by Angel Paez]   ...   
1  FBIS4-56863  d53i                               "Comrade Feliciano."   

                                      Clean_sentence  Words  Chars  \
0  BFN 106 Exclusive report angel Paez text the s...     31    190   
1                                  Comrade Feliciano      2     17   

   tf-idf_score  adverb_score  verb_score  adjective_score  text_rank_score  \
0      0.017868       0.02439    0.035714            0.016         0.013294   
1      0.004720       0.00000    0.000000            0.000         0.008805   

   font_style_score  sentence_position  thematic_score  sentence_similarity  \
0          0.032258                1.0        0.121429             0.351307   
1          0.000000                0.8        0.007143             0.601866   

   combined_score  
0        0.375351  
1        0.266357

In [ ]:
def generate_combine_summary(lst):
  dir = lst[0]
  doc = lst[1]
  temp_df = dataset[(dataset['Dir'] == dir) & (dataset['Docno'] == doc)].sort_values(by = 'combined_score', ascending = False)
  rs = ''
  cnt = 0
  for i in temp_df['sentence']:
    if cnt + len(i.split()) <= 100:
      cnt += len(i.split())
      rs += (i+' ')
  return rs
final_dataset['combined_score'] = final_dataset[['Dir', 'Docno']].apply(generate_combine_summary, axis = 1)

In [ ]:
final_dataset = pd.read_csv("/content/drive/MyDrive/project/Summary.csv")

In [ ]:
final_dataset.head(2)

Unnamed: 0           Docno   Dir  \
0           0   AP880926-0203  d30e   
1           1  WSJ910607-0063  d30e   

                                        tf-idf_score  \
0  Officials of the seven key industrialized nati...   
1  The Christian Children's Fund ofRichmond, Va.,...   

                                        adverb_score  \
0  ``It is ... sad to note that we, American, Asi...   
1  Trickle Up, which makes thousands of $100 loan...   

                                          verb_score  \
0  ``In a market place where one group of operato...   
1  David Munro, an officialwith TechnoServe of No...   

                                     adjective_score  \
0  Officials of the seven key industrialized nati...   
1     Tiny Third World businesses commonly repay ...   

                                     text_rank_score  \
0  Brazil, with $121 billion in foreign loans, ha...   
1     Tiny Third World businesses commonly repay ...   

                                    font_style_score  \
0     Third World countries led by Brazil, thewor...   
1  The Christian Children's Fund ofRichmond, Va.,...   

                                   sentence_position  \
0     Third World countries led by Brazil, thewor...   
1     Tiny Third World businesses commonly repay ...   

                                      thematic_score  \
0     Third World countries led by Brazil, thewor...   
1  Trickle Up, which makes thousands of $100 loan...   

                                 sentence_similarity  \
0     Third World countries led by Brazil, thewor...   
1  Undaunted, the women switched to chickens and ...   

                                      combined_score  
0     Third World countries led by Brazil, thewor...  
1     Tiny Third World businesses commonly repay ...